In [2]:
# Import necessary modules
import requests
from bs4 import BeautifulSoup
import numpy as np
import re

import scraping_class
logfile = 'log_jobindex_scraping.txt'
connector = scraping_class.Connector(logfile)

#Define the url and fetch the HTML using the requests module
url = 'https://www.jobindex.dk/jobsoegning?lang=da.html'
response = requests.get(url)  
html = response.text  
soup = BeautifulSoup(html,'lxml') # parse the raw html using BeautifoulSoup

#Find the pagination and page links
pagination = soup\
            .find('ul',{'class':'pagination'})\
            .find_all('a', {'class':'page-link'})
#print(pagination)

#Extract the text-part of the pagination links, i.e. not the link but the page number. 
page_list = []
for page in pagination:
    page_list.append(page.text)
    last_page = int(page_list[-1]) #Save the last page number as "last page"
print(last_page)

ConnectionError: HTTPSConnectionPool(host='www.jobindex.dk', port=443): Max retries exceeded with url: /jobsoegning?lang=da.html (Caused by NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x00000216A7D58D88>: Failed to establish a new connection: [WinError 10060] A connection attempt failed because the connected party did not properly respond after a period of time, or established connection failed because connected host has failed to respond'))

In [3]:
#Get all link pages from jobindex
jobindex_links = []
title_company = []
job_data = []
for i in range(1,3):
    url = f'https://www.jobindex.dk/jobsoegning?maxdate=20200826&mindate=20200526&jobage=archive'
    jobindex_links.append(url)   
# Get html behind each search page from jobindex
for link in jobindex_links:
    response,call_id = connector.get(link,'searchpage')
    soup = BeautifulSoup(response.text, 'lxml')
    joblistings = soup.find_all('div',{'class':'jobsearch-result'})
# Extract info for each job listing from each searchpage    
    for joblisting in joblistings:
        # find the job title and company which are in bold
        title_and_company = re.findall(r'(?<=<b>)(.*)(?=</b>)', str(joblisting)) 
        title_and_company = title_and_company[0:2]
        soup = BeautifulSoup(str(joblisting), 'lxml')
        # Indirectly extracting the short job description by removing everything else
        for div in soup.find_all("div", {'class':'jix_toolbar jix_appetizer_toolbar'}): # remove toolbar at the end of each job
            div.decompose()
        soup = soup.get_text()
        soup = soup.replace("\n", " ") # remove \n 
        job_info = [str(title_and_company[0]),str(title_and_company[1]),str(soup)] #create a list containing job title, company and description
        job_data.append(job_info)
    
#transforming into dataframe

import pandas as pd
df = pd.DataFrame(job_data)
df.columns = ["Job Title", "Company", "Description"]


# remove job title and company info from description column
df['Description'] = df['Description'].replace(to_replace=r'\b'+df['Job Title']+r'\b', value='',regex=True)
df['Description'] = df['Description'].replace(to_replace=r'\b'+df['Company']+r'\b', value='',regex=True)
df['Description'] = df['Description'].str.strip()

IndexError: list index out of range

In [13]:
import nltk
df['Tokenized_description'] = ' '
symbols = ['?','!','>','<','-','[',']','(',')','{','}',' –','``',"''",'""','\\','@','$','&','=']
for i in range(len(df)):
    stop_words = nltk.corpus.stopwords.words("danish")
    df['Tokenized_description'][i] = re.sub(r'(\.+ )|,|\||:|/|\'|\-|;|\*|!|(\s\d+\s)|(\s\W\s)',' ',str(df['Description'][i]))
    df['Tokenized_description'][i] = str(df['Tokenized_description'][i]).rstrip('\\')
    df['Tokenized_description'][i] = nltk.word_tokenize(str(df['Tokenized_description'][i].lower()))
    df['Tokenized_description'][i] = [w for w in df['Tokenized_description'][i] if not w in stop_words] 
    df['Tokenized_description'][i] = [w for w in df['Tokenized_description'][i] if not w in symbols]
    df['Tokenized_description'][i] = [w for w in df['Tokenized_description'][i] if not w.isnumeric()]
    df['Tokenized_description'][i] = [word for word in df['Tokenized_description'][i] if len(word) > 3]
    
descriptions_list = [] 
# This loop will pull all tokens in one bag
for i in range(len(df)):
    descriptions_list.extend(df['Tokenized_description'][i]) 
    
word_dist = nltk.FreqDist(descriptions_list)
word_dist.most_common(10)

[('vores', 40),
 ('søger', 26),
 ('arbejde', 22),
 ('samt', 18),
 ('opgaver', 18),
 ('samarbejde', 14),
 ('børn', 14),
 ('stilling', 14),
 ('udvikling', 12),
 ('derfor', 12)]